# Extraction and insertion

In this notebook there will be only the **final, necessary** code for the data extraction (cleaning/filtering/merging) and data import.

In [25]:
import pandas as pd
import os 
import numpy as np
import re

## Extraction

In [49]:
rcp = pd.read_csv("ResearchCollectionPublications2008_2018.tsv", sep="\t", header=0, encoding="latin-1") #utf8 not working...

### 1. Filtering 

In [27]:
# 1. type of publication only limited to: Journal Article, Conference Paper, Other Conference Item, Book Chapter
rcp_tf = rcp.loc[(rcp["DC_TYPE"] == "Journal Article")|
                (rcp["DC_TYPE"] == "Conference Paper")|
                (rcp["DC_TYPE"] =="Other Conference Item")|
                (rcp["DC_TYPE"] =="Book Chapter"),:]

In [28]:
#2. projecting into [id, author, date, title, type, journal title]
rcp_ff = rcp_tf[["RC_ID","DC_CONTRIBUTOR_AUTHOR","DC_DATE_ISSUED","DC_TITLE","DC_TYPE","ETHZ_JOURNAL_TITLE"]]

In [29]:
rcp_ff.head()

,RC_ID,DC_CONTRIBUTOR_AUTHOR,DC_DATE_ISSUED,DC_TITLE,DC_TYPE,ETHZ_JOURNAL_TITLE
97,3379,"Gonzalez-Nicolini, Valeria||Fussenegger, Martin",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
98,15103,"Vorrath, Judith",2008,From refugee to reintegration crisis?,Book Chapter,L'Afrique des grands lacs
99,158533,"Burri, Regula Valérie||Dumit, Joseph",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN
100,14698,"Glaser, Marie A.",2008,Die Baustelle,Book Chapter,NaN
101,8255,"Knubel, Denis||Greenwood, Greg||Wiegandt, Ellen",2008,Research and development in mountain glaciers,Book Chapter,NaN


In [30]:
#3. changing column names ro more readable ones
rcp_ff = rcp_ff.rename({"RC_ID": "id", "DC_CONTRIBUTOR_AUTHOR": "author", "DC_DATE_ISSUED": "publication date",
                       "DC_TITLE": "title", "DC_TYPE": "publication type", "ETHZ_JOURNAL_TITLE": "journal"}, 
              axis = 1)
rcp_ff.head()


,id,author,publication date,title,publication type,journal
97,3379,"Gonzalez-Nicolini, Valeria||Fussenegger, Martin",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
98,15103,"Vorrath, Judith",2008,From refugee to reintegration crisis?,Book Chapter,L'Afrique des grands lacs
99,158533,"Burri, Regula Valérie||Dumit, Joseph",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN
100,14698,"Glaser, Marie A.",2008,Die Baustelle,Book Chapter,NaN
101,8255,"Knubel, Denis||Greenwood, Greg||Wiegandt, Ellen",2008,Research and development in mountain glaciers,Book Chapter,NaN


### 2. Cleaning

In [31]:
# 1. Cleaning the names 

In [32]:
def separate_names(names):
    """ Separes a string of names of the form name1||name2||name3||... into a list of names."""
    # Also: does pandas DataFrame support list of strings as fields? 
    # Andreas: Yes
    if isinstance(names, str):
        return names.split("||")

In [33]:
author_array = rcp_ff["author"].apply(separate_names)
rcp_ff["author"] = author_array
print("Check that we don't have more missing values: ", author_array.shape[0] - np.count_nonzero(author_array))
rcp_ff.head()

Check that we don't have more missing values:  8


,id,author,publication date,title,publication type,journal
97,3379,"[Gonzalez-Nicolini, Valeria, Fussenegger, Martin]",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
98,15103,"[Vorrath, Judith]",2008,From refugee to reintegration crisis?,Book Chapter,L'Afrique des grands lacs
99,158533,"[Burri, Regula Valérie, Dumit, Joseph]",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN
100,14698,"[Glaser, Marie A.]",2008,Die Baustelle,Book Chapter,NaN
101,8255,"[Knubel, Denis, Greenwood, Greg, Wiegandt, Ellen]",2008,Research and development in mountain glaciers,Book Chapter,NaN


In [34]:
rcp_ff_e = rcp_ff.explode("author")
print(rcp_ff.shape, rcp_ff_e.shape)

(98367, 6) (521431, 6)


In [35]:
rcp_ff_e.head()

,id,author,publication date,title,publication type,journal
97,3379,"Gonzalez-Nicolini, Valeria",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
97,3379,"Fussenegger, Martin",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
98,15103,"Vorrath, Judith",2008,From refugee to reintegration crisis?,Book Chapter,L'Afrique des grands lacs
99,158533,"Burri, Regula Valérie",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN
99,158533,"Dumit, Joseph",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN


In [36]:
# 2. Cleaning the dates (same formatting)

In [37]:
def date_to_year(date):
    "Get the dates into a single format (YYYY)"
    if isinstance(date, str):
        return re.findall("[\d]{4}",date)[0]

def date_to_year_and_month(date):
    "Get dates into format YYYY-MM"
    # if needed later
    if isinstance(date, str):
        if len(re.findall("[\d]{4}[-][\d]{2}",date)) > 0:
            return re.findall("[\d]{4}[-][\d]{2}",date)[0]
        else:
            return str(date)

In [38]:
print("Missing values before: ", rcp_ff_e["publication date"].shape[0] - np.count_nonzero(rcp_ff_e["publication date"]))
date_array = rcp_ff_e["publication date"].apply(date_to_year)
rcp_ff_e["publication date"] = date_array
print("Missing values after: ", author_array.shape[0] - np.count_nonzero(author_array))
rcp_ff_e.head()

Missing values before:  0
Missing values after:  8


,id,author,publication date,title,publication type,journal
97,3379,"Gonzalez-Nicolini, Valeria",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
97,3379,"Fussenegger, Martin",2008,Adenovirus-mediated transduction of auto- and ...,Book Chapter,Methods in Molecular Biology
98,15103,"Vorrath, Judith",2008,From refugee to reintegration crisis?,Book Chapter,L'Afrique des grands lacs
99,158533,"Burri, Regula Valérie",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN
99,158533,"Dumit, Joseph",2008,Social Studies of Scientific Imaging and Visua...,Book Chapter,NaN


### 3. Merging

#### 3.1 Organisations data

In [39]:
pl = pd.read_excel("ETH Professor list.xlsx")

In [40]:
# create professor column and keep only the relevant columns
pl["Professor"] = pl["Name"] + ", " + pl["First name"]
pl = pl[["Professor", "Org. unit code", "Organisation"]]
pl.head()

,Professor,Org. unit code,Organisation
0,"Abhari, Reza S.",2627,Institute of Energy Technology (former)
1,"Acciaio, Beatrice",9727,"Acciaio, Beatrice"
2,"Ackermann, Martin",3743,"Ackermann, Martin"
3,"Ackermann, Martin",2721,Inst. Biogeochem. and Pollutant Dynamics
4,"Adey, Bryan T.",2604,Inst. Construction&Infrastructure Manag.


In [96]:
rc_m = pl.merge(rcp_ff_e,how="outer",left_on="Professor",right_on="author")
rc_m.head(3)

,Professor,Org. unit code,Organisation,id,author,publication date,title,publication type,journal
0,"Abhari, Reza S.",2627.0,Institute of Energy Technology (former),253873.0,"Abhari, Reza S.",2008,The Dynamics of the Vorticity Field in a Low S...,Other Conference Item,NaN
1,"Abhari, Reza S.",2627.0,Institute of Energy Technology (former),13706.0,"Abhari, Reza S.",2008,Aerothermal Performance of Streamwise and Comp...,Conference Paper,NaN
2,"Abhari, Reza S.",2627.0,Institute of Energy Technology (former),16637.0,"Abhari, Reza S.",2008,Unsteady CFD Investigation on Inlet Distortion...,Conference Paper,Proceedings of the ASME Turbo Expo


In [81]:
rc_m.shape

(527256, 9)

In [82]:
rcp_ff_e.shape

(521431, 6)

#### 3.2 Research areas data

In [50]:
ar = pd.read_excel("areas.xls")

In [51]:
ar.head(3)

,ANREDE,FAMNAME,VORNAME,DEPARTEMENT_NAME,DEPARTEMENT_LEITZAHL,FORSCHUNGSGEBIET_E,HOMEPAGE
0,Herr,Avermaete,Tom,Architektur,2100,NaN,NaN
1,Herr,Block,Philippe Camille Vincent,Architektur,2100,<p>Philippe Block is a structural engineer and...,http://block.arch.ethz.ch
2,Herr,Brandlhuber,Arno Hans,Architektur,2100,NaN,NaN


In [66]:
ar.shape
# Only 564 researchers ... 
# Hypothesis of work: select the 'most complete' subset

(564, 8)

In [55]:
# small adjustment 
ar.loc[ar["ANREDE"]=="Herr",["ANREDE"]] = "M"
ar.loc[ar["ANREDE"]=="Frau",["ANREDE"]] = "F"

In [57]:
# producing single name column 
ar["Name"] = ar["FAMNAME"] + ", " + ar["VORNAME"]

In [ ]:
# Should we translate the department names? 
# (Check how they're represented in other files)

In [69]:
# projecting into interesting columns 
# Should we keep the "FORSCHUNGSGEBIET_E" column? 396 values are NaN (circa 70%)
# same goes for the Homepage (btw 60-70% NaN)
ar_p = ar[["ANREDE","DEPARTEMENT_NAME","DEPARTEMENT_LEITZAHL","FORSCHUNGSGEBIET_E","HOMEPAGE","Name"]]

In [70]:
# Renaming the columns 
ar_p.columns = ["gender","department_name","department_code","description","webpage","name"]
ar_p.head(3)

,gender,department_name,department_code,description,webpage,name
0,M,Architektur,2100,NaN,NaN,"Avermaete, Tom"
1,M,Architektur,2100,<p>Philippe Block is a structural engineer and...,http://block.arch.ethz.ch,"Block, Philippe Camille Vincent"
2,M,Architektur,2100,NaN,NaN,"Brandlhuber, Arno Hans"


In [97]:
# finally merging
rc_m = ar_p.merge(rc_m,how="outer",left_on="name",right_on="author")
rc_m.head(3)

,gender,department_name,department_code,description,webpage,name,Professor,Org. unit code,Organisation,id,author,publication date,title,publication type,journal
0,M,Architektur,2100.0,NaN,NaN,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom",284605.0,"Avermaete, Tom",2017,"Death of the Author, Center and Meta-Theory: E...",Book Chapter,NaN
1,M,Architektur,2100.0,NaN,NaN,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom",288639.0,"Avermaete, Tom",2018,Balcony,Book Chapter,NaN
2,M,Architektur,2100.0,NaN,NaN,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom",284909.0,"Avermaete, Tom",2018,The View from the Grid,Book Chapter,NaN


In [79]:
rc_m.shape
# we gained about 250 researchers from the outer join ... 

(527498, 15)

#### 3.3 Selection

In [176]:
rc_m.groupby("department_name").size().sort_values(ascending=False)

department_name
Maschinenbau und Verfahrenstechnik            3985
Physik                                        3945
Informationstechnologie und Elektrotechnik    3108
Chemie und Angewandte Biowissenschaften       2841
Umweltsystemwissenschaften                    2783
Bau, Umwelt und Geomatik                      2466
Gesundheitswissenschaften und Technologie     1951
Management, Technologie und Ökonomie          1354
Biologie                                      1336
Informatik                                    1106
Geistes-, Sozial- und Staatswissenschaften    1080
Biosysteme                                    1002
Mathematik                                     860
Erdwissenschaften                              654
Materialwissenschaft                           506
Architektur                                    348
dtype: int64

In [154]:
titlexdep = pd.DataFrame(rc_m[["department_code","title"]].groupby("title").count())

In [159]:
titlexdep= titlexdep.reset_index()

In [160]:
titlexdep.columns = ["title","num_dept"]

In [166]:
temp_df = titlexdep.merge(rc_m,how="right",left_on="title",right_on="title")

In [174]:
temp_df[["department_name","num_dept"]].groupby("department_name").mean().sort_values(by="num_dept",ascending=False)

,num_dept
department_name,
Physik,2.414907
"Geistes-, Sozial- und Staatswissenschaften",2.015858
Materialwissenschaft,1.686747
Erdwissenschaften,1.588509
Maschinenbau und Verfahrenstechnik,1.492692
Umweltsystemwissenschaften,1.410591
Biosysteme,1.383065
Mathematik,1.328588
Gesundheitswissenschaften und Technologie,1.317994


In [165]:
rc_m.isnull().sum(axis=0)

gender              498173
department_name     498173
department_code     498173
description         516561
webpage             514630
name                498173
Professor           479872
Org. unit code      479872
Organisation        479872
id                     360
author                 368
publication date      1593
title                  360
publication type       360
journal              83200
is_professor             0
dtype: int64

#### 3.4 Final processing 

In [98]:
# creating boolean column to signal if the author is a professor 
rc_m["is_professor"] = rc_m["author"]==rc_m["Professor"]
rc_m.head(3)

,gender,department_name,department_code,description,webpage,name,Professor,Org. unit code,Organisation,id,author,publication date,title,publication type,journal,is_professor
0,M,Architektur,2100.0,NaN,NaN,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom",284605.0,"Avermaete, Tom",2017,"Death of the Author, Center and Meta-Theory: E...",Book Chapter,NaN,True
1,M,Architektur,2100.0,NaN,NaN,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom",288639.0,"Avermaete, Tom",2018,Balcony,Book Chapter,NaN,True
2,M,Architektur,2100.0,NaN,NaN,"Avermaete, Tom","Avermaete, Tom",9643.0,"Avermaete, Tom",284909.0,"Avermaete, Tom",2018,The View from the Grid,Book Chapter,NaN,True


In [ ]:
# drop the excessive columns

In [43]:
rc_m.columns = ["organisation_code","organisation_name","publication_id",
                "person_name","publication_date","publication_title","publication_type",
                "publication_journal", "person_is_professor"]
rc_m.head(3)

,organisation_code,organisation_name,publication_id,person_name,publication_date,publication_title,publication_type,publication_journal,person_is_professor
0,2627.0,Institute of Energy Technology (former),253873.0,"Abhari, Reza S.",2008,The Dynamics of the Vorticity Field in a Low S...,Other Conference Item,NaN,True
1,2627.0,Institute of Energy Technology (former),13706.0,"Abhari, Reza S.",2008,Aerothermal Performance of Streamwise and Comp...,Conference Paper,NaN,True
2,2627.0,Institute of Energy Technology (former),16637.0,"Abhari, Reza S.",2008,Unsteady CFD Investigation on Inlet Distortion...,Conference Paper,Proceedings of the ASME Turbo Expo,True


#### Wierd missing values

In [44]:
# TODO : look into the null names 
# - where does this come from? 
sum(rc_m["person_name"].isnull())
# My suspicion is that 118/126 missing authors correspond to 
# 'organisation' rows that survived the outer join (meaning that 
# those are organisations which are not linked to any publication), 
# while we actually have only 8 missing authors 

126

In [45]:
# This mistery is solved: 118 entries have null publication values 
# because of the outer join -> it's entries about organisations which do 
# not have any publications associated! 

# sum(rc_m["publication_type"].isnull())
# sum(rc_m["publication_id"].isnull()) 
# sum(rc_m["publication_title"].isnull())

In [46]:
# this is the most plausible- but still we should check the information
# was missing in the original file as well
sum(rc_m["publication_date"].isnull())

1351

## Import/export :):

In [47]:
rc_m.to_csv("metadata_final.csv", index=False)
rc_m.to_json("metadata_final.json", orient = "records")

### Neo4j import details

The following nodes will be created: 
- **person** [name, professor]
- **publication** [id, title, date, type, journal]
- **organisation** [name, code]


--- 


**The commands**

    
To load the csv you first have to <u>copy it into your Neo4j base directory</u>. More info [here](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/#load-csv-import-data-from-a-csv-file). I did the following: 

```cp .\metadata_final.csv C:/Users/Giulia/.Neo4jDesktop/neo4jDatabases/database-befe90d3-7991-457e-9671-62c55c830654/installation-3.5.12/import```

<u>Constraints first</u>

The constraints are here to make sure we don't create duplicate nodes.

    CREATE CONSTRAINT ON (c:Person) ASSERT c.name IS UNIQUE;
    CREATE CONSTRAINT ON (c:Organisation) ASSERT c.name IS UNIQUE;
    CREATE CONSTRAINT ON (c:Publication) ASSERT c.title IS UNIQUE;

 
Now we'll <u>load the data</u> in a very lightweight manner: 

1) person nodes <br>
```
    LOAD CSV WITH HEADERS FROM "file:///metadata_final.csv" AS line
    WITH line WHERE line.person_name IS NOT NULL
    MERGE (person:Person {name:line.person_name, is_professor:line.person_is_professor});
```
        > Added 176604 labels, created 176604 nodes, set 353208 properties, completed after 8880 ms.

2) publication nodes (this might take a while) <br>
       
        LOAD CSV WITH HEADERS FROM "file:///metadata_final.csv" AS line
        WITH line where line.publication_id IS NOT NULL
        MERGE (publication: Publication {title: line.publication_title})
        SET publication.id= line.publication_id,            
            publication.journal=line.publication_journal, 
            publication.type=line.publication_type, 
            publication.date=date(line.publication_date);

        > Added 96014 labels, created 96014 nodes, set 2121683 properties, completed after 9349 ms.
        
3) organisation nodes <br> 
    
        LOAD CSV WITH HEADERS FROM "file:///metadata_final.csv" AS line
        WITH line where line.organisation_code IS NOT NULL
        MERGE (organisation:Organisation {name:line.organisation_name, 
                                          code:line.organisation_code});
                                          
        > Added 383 labels, created 383 nodes, set 766 properties, completed after 1822 ms.                          
            
        
4) finally all the edges <br> 
        
        LOAD CSV WITH HEADERS FROM "file:///metadata_final.csv" AS line
        MATCH (person:Person {name:line.person_name}), 
               (publication:Publication {id:line.publication_id})
        MERGE (person)-[:PUBLISHED]->(publication)
        MERGE (publication)-[:AUTHOR]->(person);
        
        LOAD CSV WITH HEADERS FROM "file:///metadata_final.csv" AS line
        MATCH (person:Person {name:line.person_name}),
               (organisation:Organisation {code:line.organisation_code})
        MERGE (person)-[:BELONGS_TO]->(organisation)
        MERGE (organisation)-[:CONTAINS]->(person);
        
Note: in case you did something wrong and you want to erase the network here's the query: 

        MATCH (n)
        DETACH DELETE n;

    


In [48]:
# question: is it correct to say that the title is a unique identifier of the paper? 
# same goes for organisation/author names